# RO System Simulation Engine with MCAS

This notebook serves as a simulation engine for RO systems using the MCAS property package,
leveraging modular utility functions for maintainability and consistency.

In [ ]:
# Parameters cell - will be replaced by papermill
project_root = "/path/to/project"  # Will be replaced by papermill
configuration = {}  # RO configuration from optimization
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
membrane_properties = None
optimize_pumps = False
feed_ion_composition = None  # JSON string with ion concentrations in mg/L
initialization_strategy = "sequential"  # "sequential", "block_triangular", "custom_guess", "relaxation"

## Setup and Imports

In [ ]:
# System imports
import sys
import os
import json
import warnings
from pathlib import Path

# Add project root to path
sys.path.insert(0, project_root)

# Import all utility modules for MCAS simulation
from utils.mcas_builder import (
    build_mcas_property_configuration,
    check_electroneutrality,
    get_total_dissolved_solids,
    calculate_ionic_strength
)
from utils.ro_model_builder import build_ro_model_mcas
from utils.ro_initialization import initialize_multistage_ro_elegant
from utils.ro_solver import initialize_and_solve_mcas
from utils.ro_results_extractor import extract_results_mcas
from utils.response_formatter import format_simulation_response
from utils.config import get_config
from utils.logging_config import get_configured_logger

# Configure logging
logger = get_configured_logger(__name__)
warnings.filterwarnings('ignore')

# Results storage
results = {}

## Validate and Prepare Ion Composition

In [ ]:
# Validate configuration
if not isinstance(configuration, dict) or "stages" not in configuration:
    raise ValueError("Invalid configuration provided")

# Parse and validate ion composition (required for MCAS)
if not feed_ion_composition:
    raise ValueError("Ion composition is required for MCAS simulation")

if isinstance(feed_ion_composition, str):
    parsed_ion_composition = json.loads(feed_ion_composition)
else:
    parsed_ion_composition = feed_ion_composition

# Basic validation of ion composition
if not isinstance(parsed_ion_composition, dict):
    raise ValueError("Ion composition must be a dictionary")

for ion, conc in parsed_ion_composition.items():
    if not isinstance(conc, (int, float)) or conc < 0:
        raise ValueError(f"Invalid concentration for {ion}: {conc}")

# Add feed_flow_m3h to configuration if missing
if "feed_flow_m3h" not in configuration:
    if "recycle_info" in configuration and "effective_feed_flow_m3h" in configuration["recycle_info"]:
        configuration["feed_flow_m3h"] = configuration["recycle_info"]["effective_feed_flow_m3h"]
    elif "stages" in configuration and configuration["stages"]:
        first_stage = configuration["stages"][0]
        if "feed_flow_m3h" in first_stage:
            configuration["feed_flow_m3h"] = first_stage["feed_flow_m3h"]
        else:
            configuration["feed_flow_m3h"] = 100.0
    else:
        configuration["feed_flow_m3h"] = 100.0

print(f"Configuration: {configuration.get('array_notation', 'Unknown')}")
print(f"Feed flow: {configuration['feed_flow_m3h']} m³/h")
print(f"Feed salinity: {feed_salinity_ppm} ppm")
print(f"Feed temperature: {feed_temperature_c}°C")
print(f"Initialization strategy: {initialization_strategy}")

## Build MCAS Property Configuration

In [ ]:
# Build MCAS property configuration from ion composition
mcas_config = build_mcas_property_configuration(
    feed_composition=parsed_ion_composition,
    include_scaling_ions=True,
    include_ph_species=True
)

# Display ion composition analysis
print("\nFeed Ion Composition (mg/L):")
for ion, conc in sorted(parsed_ion_composition.items()):
    print(f"  {ion:8s}: {conc:8.1f}")

# Check electroneutrality
is_neutral, imbalance = check_electroneutrality(parsed_ion_composition)
if not is_neutral:
    print(f"\nWarning: Charge imbalance of {imbalance:.1%}")
else:
    print("\nCharge balance: OK")

# Calculate properties
actual_tds = get_total_dissolved_solids(parsed_ion_composition)
ionic_strength = calculate_ionic_strength(parsed_ion_composition)

print(f"\nTotal TDS: {actual_tds:.0f} mg/L")
print(f"Ionic strength: {ionic_strength:.3f} mol/L")
print(f"\nMCAS components: {', '.join(mcas_config['solute_list'])}")

## Build and Initialize Model

In [ ]:
# Build the RO model with MCAS
logger.info("Building RO model with MCAS property package...")

model = build_ro_model_mcas(
    config_data=configuration,
    mcas_config=mcas_config,
    feed_salinity_ppm=feed_salinity_ppm,
    feed_temperature_c=feed_temperature_c,
    membrane_type=membrane_type,
    membrane_properties=membrane_properties
)

print(f"Model built successfully with {configuration['stage_count']} stages")

## Solve Model

In [ ]:
# Initialize and solve the model
try:
    logger.info(f"Initializing and solving with strategy: {initialization_strategy}")
    
    # Use the centralized solver function
    solver_results = initialize_and_solve_mcas(
        model=model,
        config_data=configuration,
        optimize_pumps=optimize_pumps
    )
    
    if solver_results['status'] == 'success':
        print("\nSolution found successfully!")
        print(f"Solver time: {solver_results.get('solve_time', 0):.1f} seconds")
    else:
        raise RuntimeError(f"Solver failed: {solver_results.get('message', 'Unknown error')}")
        
except Exception as e:
    logger.error(f"Solve failed: {str(e)}", exc_info=True)
    results = {
        "status": "error",
        "message": f"Simulation failed during solve: {str(e)}",
        "error_type": type(e).__name__
    }
    raise

## Extract and Format Results

In [ ]:
# Extract results from the solved model
if solver_results['status'] == 'success':
    logger.info("Extracting results...")
    
    # Use the centralized results extractor
    raw_results = extract_results_mcas(model, configuration)
    
    # Format results for output
    results = format_simulation_response(raw_results)
    
    # Add metadata
    results["property_package"] = "MCAS"
    results["initialization_strategy"] = initialization_strategy
    results["solver_info"] = {
        "termination_condition": solver_results.get('termination_condition', 'optimal'),
        "solve_time": solver_results.get('solve_time', 0),
        "iterations": solver_results.get('iterations', 0)
    }
    
    logger.info("Results extraction complete")
        
else:
    results = {
        "status": "error",
        "message": f"Solver did not succeed: {solver_results.get('message', 'Unknown error')}"
    }

## Display Results Summary

In [ ]:
# Display comprehensive results summary
if 'results' in locals() and results.get("status") == "success":
    print("\n" + "="*60)
    print("SIMULATION RESULTS - MCAS MODEL")
    print("="*60)
    
    # Get the actual results data (nested under 'results' key after formatting)
    sim_data = results.get('results', {})
    
    # Overall performance
    if 'performance' in sim_data:
        perf = sim_data['performance']
        print(f"\nOverall Performance:")
        print(f"  Available keys: {list(perf.keys())}")
        
        # Use flexible key names
        recovery_key = 'system_recovery' if 'system_recovery' in perf else 'total_recovery'
        feed_flow_key = 'feed_flow_m3h' if 'feed_flow_m3h' in perf else 'feed_flow'
        perm_flow_key = 'permeate_flow_m3h' if 'permeate_flow_m3h' in perf else 'permeate_flow'
        perm_tds_key = 'permeate_tds_ppm' if 'permeate_tds_ppm' in perf else 'permeate_tds'
        
        if recovery_key in perf:
            print(f"  Total Recovery: {perf[recovery_key]:.1%}")
        if feed_flow_key in perf:
            print(f"  Feed Flow: {perf[feed_flow_key]:.1f} m³/h")
        if perm_flow_key in perf:
            print(f"  Permeate Flow: {perf[perm_flow_key]:.1f} m³/h")
        if perm_tds_key in perf:
            print(f"  Permeate TDS: {perf[perm_tds_key]:.0f} ppm")
    
    # Economics
    if 'economics' in sim_data:
        econ = sim_data['economics']
        print(f"\nEconomics:")
        print(f"  Available keys: {list(econ.keys())}")
        
        if 'specific_energy_kwh_m3' in econ:
            print(f"  Specific Energy: {econ['specific_energy_kwh_m3']:.2f} kWh/m³")
        if 'total_power_kw' in econ:
            print(f"  Total Power: {econ['total_power_kw']:.1f} kW")
        if 'lcow_usd_m3' in econ:
            print(f"  LCOW: ${econ['lcow_usd_m3']:.2f}/m³")
        if 'capital_cost_usd' in econ:
            print(f"  CAPEX: ${econ['capital_cost_usd']:,.0f}")
        if 'annual_opex_usd' in econ:
            print(f"  Annual OPEX: ${econ['annual_opex_usd']:,.0f}/year")
    
    # Ion rejection
    if "ion_tracking" in sim_data and "overall_rejection" in sim_data["ion_tracking"]:
        print(f"\nIon Rejection:")
        for ion, rejection in sorted(sim_data['ion_tracking']['overall_rejection'].items()):
            print(f"  {ion:8s}: {rejection:.1%}")
    
    # Stage details
    if 'stage_results' in sim_data:
        print(f"\nStage Details:")
        stages = sim_data['stage_results']
        if isinstance(stages, list):
            for i, stage in enumerate(stages, 1):
                print(f"\n  Stage {i} keys: {list(stage.keys())}")
                print(f"  Stage {i}:")
                
                # Use flexible key access
                if 'feed_pressure_bar' in stage:
                    print(f"    Feed pressure: {stage['feed_pressure_bar']:.1f} bar")
                if 'water_recovery' in stage:
                    print(f"    Water recovery: {stage['water_recovery']:.1%}")
                if 'permeate_flow_m3h' in stage:
                    print(f"    Permeate flow: {stage['permeate_flow_m3h']:.1f} m³/h")
                if 'permeate_tds_ppm' in stage:
                    print(f"    Permeate TDS: {stage['permeate_tds_ppm']:.0f} ppm")
                if 'pump_power_kw' in stage:
                    print(f"    Pump power: {stage['pump_power_kw']:.1f} kW")
    
    # Mass balance check
    if "mass_balance" in sim_data:
        mb = sim_data["mass_balance"]
        print(f"\nMass Balance:")
        if 'error_percent' in mb:
            print(f"  Error: {mb['error_percent']:.2f}%")
            if mb['error_percent'] > 0.1:
                print("  WARNING: Mass balance error exceeds 0.1%")
            else:
                print("  Mass balance OK")
                
elif 'results' in locals():
    print(f"\nSimulation completed but results status: {results.get('status', 'unknown')}")
    print(f"Message: {results.get('message', 'No message')}")
    if 'results' in results:
        print(f"Available sections: {list(results['results'].keys())}")
else:
    print("\nNo results available - check previous cells for errors")

## Export Results

In [ ]:
# Results cell - tagged for papermill to extract
# This cell contains the complete results dictionary
results

## Save Results to JSON

In [ ]:
# Save results to JSON file for archival
if results.get("status") == "success":
    import json
    from datetime import datetime
    
    # Create results directory if it doesn't exist
    results_dir = Path(project_root) / "results"
    results_dir.mkdir(exist_ok=True)
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = results_dir / f"simulation_results_{timestamp}.json"
    
    # Save results
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2, default=str)
    
    print(f"\nResults saved to: {output_file}")